In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
def GenerateData(filepath):
    file = pd.read_csv(filepath)
    
    imageDataset = []
    
    if 'train' in filepath:
        labelDataset = file.loc[:,'label'].values.tolist()
    else:
        labelDataset = []
    
    for i in range(len(file)):
        start = 1
        if 'test' in filepath:
            start = 0
        
        currentLine = file.loc[i]
        image = np.array(currentLine[start:])
        
        image = image.reshape(28,28)
        image = image/255
        
        imageDataset.append(image)
        
    imageDataset = np.array(imageDataset)
    labelDataset = np.array(labelDataset)
        
    imageDataset = imageDataset.reshape(imageDataset.shape[0], imageDataset.shape[1], imageDataset.shape[2], 1)
    labelDataset = labelDataset.reshape(labelDataset.shape[0],1)
        
    print(imageDataset.shape)
    print(labelDataset.shape)
        
    return imageDataset, labelDataset
    
trainImages, trainLabels = GenerateData(r'/kaggle/input/digit-recognizer/train.csv')

In [4]:
import matplotlib.pyplot as plt
import random

fig = plt.figure(figsize=(10, 10))
#rd.seed(None)
for i in range(16):
    fig.add_subplot(4, 4, i+1)
    rdNumber = random.randint(0,42000)
    plt.title(trainLabels[rdNumber][0])
    plt.imshow(trainImages[rdNumber], cmap='Greys_r')

In [5]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(trainLabels)
print(y_train)
print(trainLabels.squeeze())

In [6]:
x_train = trainImages

In [7]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

In [8]:
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

In [9]:
model = Sequential()
model.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))


model.summary()

In [10]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [12]:
history = model.fit(datagen.flow(x_train,y_train, batch_size=64), epochs = 10, shuffle=True)

In [14]:
testImages, testLabels = GenerateData(r'/kaggle/input/digit-recognizer/test.csv')

In [15]:
x_test = testImages
y_test = testLabels

In [16]:
predictions = model.predict(x_test, verbose=2)
predictions

In [18]:
predictionsLabel = np.argmax(predictions, axis=1)

In [19]:
df = pd.DataFrame({'ImageId': [i for i in range(1,len(predictionsLabel)+1)], 'Label': predictionsLabel})
df

In [20]:
df.to_csv('/kaggle/working/predictions.csv', index=False)

In [21]:
model.save('/kaggle/working')